## code functions

In [4]:
import google.generativeai as genai
import re
import json
import os
from dotenv import load_dotenv
import anthropic
load_dotenv()

True

In [5]:
genai.configure(api_key=os.environ['GEMINI-API'])
client = anthropic.Anthropic(api_key=os.environ['ANTHROPIC-API'])

In [6]:
def clean_json_response(raw_text):
    # Strip ```json ... ``` or ``` ... ``` blocks if they exist
    cleaned = re.sub(r"^```(?:json)?\s*", "", raw_text.strip())
    cleaned = re.sub(r"\s*```$", "", cleaned)
    return cleaned

In [7]:
def get_lab_questions(objectives: list[str], no: int, lang: str):
    formatted_objectives = "\n".join([f"{i+1}. {obj}" for i, obj in enumerate(objectives)])
    prompt = f"""
        You are a programming instructor generating structured lab questions.
        
        Generate exactly {no} structured programming questions in {lang} that cover the objectives given below:.
        Some questions can cover more than one objective, but make sure each objective is covered in at least one question. The questions should vary in difficulty and complexity, and should test the student's ability to apply concepts from the listed objectives effectively.

        Language: {lang}  
        Number of Questions per Objective: {no}  

        Objectives:
        {formatted_objectives}

        Generate exactly {no} questions. Each question must contain:
        - number (int)
        - title (str)
        - problem_statement (str)
        - input_format (str)
        - output_format (str)
        - sample_input (str)
        - sample_output (str)

        Return the entire result as a JSON object following this format:

        objective: str
        language: str
        questions: List[Question]
        """
    model = genai.GenerativeModel('gemini-1.5-flash')  # or 'gemini-1.5-pro'
    response = model.generate_content(prompt,generation_config=genai.GenerationConfig(response_mime_type="application/json"))
    raw_text = response.text
    cleaned_json = clean_json_response(raw_text)

    try:
        json_data = json.loads(cleaned_json)
        return json_data
    except Exception as e:
        print("Parsing Error:", e)
        print("Raw output:", raw_text)
        return None

In [9]:
lab = get_lab_questions(["Learn to write and run simple programs", "Understand the concept of variables and assignments", "Learn to use conditional statements (if/else)"], 3, "python")

In [10]:
lab

{'objective': 'Learn to write and run simple programs, Understand the concept of variables and assignments, Learn to use conditional statements (if/else)',
 'language': 'python',
 'questions': [{'number': 1,
   'title': 'Simple Calculator',
   'problem_statement': 'Write a program that performs basic arithmetic operations (addition, subtraction, multiplication, division) based on user input.  The program should take two numbers and an operator (+, -, *, /) as input and print the result.',
   'input_format': 'Three inputs in a single line: number1 operator number2 (e.g., 10 + 5)',
   'output_format': 'The result of the operation. If division by zero is attempted, print an error message.',
   'sample_input': '10 + 5',
   'sample_output': '15'},
  {'number': 2,
   'title': 'Even or Odd Checker',
   'problem_statement': 'Write a program that checks if a given number is even or odd using conditional statements. The program should take an integer as input and print whether it is even or odd.

In [12]:
def get_followup_question(question_details: dict, student_code: str):
    prompt = f"""
    You are a programming instructor evaluating a student's understanding of a specific programming concept.

    Below is the original question given to the student and the code they submitted:

    Original Question:
    {question_details}

    Student's Code:
    {student_code}

    Your task is to generate a *follow-up question* that directly builds on the student's submitted code. The goal is to test whether the student truly understood the concept by asking them to *modify, extend, or refactor* their existing code.

    DO NOT generate a completely new or unrelated problem.

    ---

    Here are some examples of what good follow-up questions might look like:

    Original Question: Write a function to calculate the factorial of a number using recursion.

    Possible Follow-Ups:
    1. *Refactoring*: Change the code to use while loop instead of for loop.
    2. *New Constraint*: Rewrite the function to handle negative inputs with appropriate error handling.
    3. *Change of Input*: Modify the function to calculate the factorial for a list of numbers.
    4. *Output Format Change*: Return the result as a formatted string: “Factorial of 5 is 120”.

    In your follow-up question, you should try to apply a similar idea—take the *same problem context, and **add a twist* that makes the student update the code.

    ---

    Return your final result in the following JSON format as a json object  (no explanation, no markdown, just the raw JSON object):
    {{
        "number": (int),
        "title": (str),
        "problem_statement": (str),
        "input_format": (str),
        "output_format": (str),
        "sample_input": (str),
        "sample_output": (str)
    }}
    """
    response = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1024,
        temperature=0.1,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    cleaned_json = clean_json_response(response.content[0].text)

    try:
        json_data = json.loads(cleaned_json)
        # If Gemini wraps the content in a top-level "LabQuestions" key, unwrap it:
        return json_data
    except Exception as e:
        print("Parsing Error:", e)
        print("Raw output:", response.content[0].text)
        return None

In [17]:
question = {'number': 3,
   'title': 'Grade Calculator with Conditional Logic',
   'problem_statement': "Write a program that calculates a student's letter grade based on their numerical score. Use the following grading scale: 90-100: A, 80-89: B, 70-79: C, 60-69: D, below 60: F. The program should take the numerical score as input and print the corresponding letter grade.",
   'input_format': 'A single integer representing the numerical score.',
   'output_format': 'The letter grade (A, B, C, D, or F).',
   'sample_input': '85',
   'sample_output': 'B'}

code = """
def calculate_grade(score):
    if 90 <= score <= 100:
        print("A")
    elif 80 <= score <= 89:
        print("B")
    elif 70 <= score <= 79:
        print("C")
    elif 60 <= score <= 69:
        print("D")
    else:
        print("F")

# Read input from the user
score = int(input())

# Call the function
calculate_grade(score)


"""
mod_question = get_followup_question(question,code)

In [18]:
mod_question

{'number': 3,
 'title': 'Grade Calculator with Plus/Minus Grades',
 'problem_statement': 'Modify the grade calculator program to include plus and minus grades. Use the following grading scale: 97-100: A+, 93-96: A, 90-92: A-, 87-89: B+, 83-86: B, 80-82: B-, 77-79: C+, 73-76: C, 70-72: C-, 67-69: D+, 63-66: D, 60-62: D-, below 60: F. The program should take the numerical score as input and print the corresponding letter grade with plus/minus if applicable.',
 'input_format': 'A single integer representing the numerical score.',
 'output_format': 'The letter grade (A+, A, A-, B+, B, B-, C+, C, C-, D+, D, D-, or F).',
 'sample_input': '91',
 'sample_output': 'A-'}

In [11]:
def get_hint(question, code, error=None):
    prompt=f"""
    You are a programming mentor helping a student who is stuck on a problem.

    Your job is to guide the student using *three progressive hints* that help them understand and fix their mistake—without giving away the answer directly.

    Here’s the problem they were trying to solve:

    {question}

    {f"The student encountered the following error:\n{error}" if error else "The student is not getting the expected output, but there is no error."}

    Here is the code written by the student:

    {code}

    Please provide *3 hints* in *increasing order of specificity*:
    - Hint 1: A broad-level nudge (conceptual)
    - Hint 2: A more focused tip (e.g., what part of the code to look at or rethink)
    - Hint 3: A detailed logic hint (almost the solution, but not code)

    Return the hints as a JSON object with this format witth no additional markdown text:
    
        "hint_1": "...",
        "hint_2": "...",
        "hint_3": "..."
    
    """
    response = client.messages.create(
        model="claude-3-opus-20240229",  # or claude-3-sonnet or haiku
        max_tokens=1024,
        temperature=0.7,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    cleaned_json = clean_json_response(response.content[0].text)
    try:
        #text = response.content[0].text.strip()
        parsed_json = json.loads(cleaned_json)
        return parsed_json
    except Exception as e:
        print("Parsing Error:", e)
        print("Raw response:", response.content[0].text)
        return None

In [ ]:
question = {'number': 1,
   'title': 'Dynamic Matrix Multiplication',
   'problem_statement': 'Write a C program that performs matrix multiplication using dynamic memory allocation (DMA).  The program should first prompt the user for the dimensions of two matrices (rows and columns for each).  It should then dynamically allocate memory for the matrices using DMA.  The program should then prompt the user to input the elements of the matrices. Finally, it should perform the matrix multiplication and display the resultant matrix.  Handle potential errors, such as incompatible matrix dimensions for multiplication, or memory allocation failures.',
   'input_format': "The first line contains two integers, 'm' and 'n', representing the rows and columns of the first matrix. The next 'm' lines contain 'n' integers representing the elements of the first matrix. The next line contains two integers, 'p' and 'q', representing the rows and columns of the second matrix. The next 'p' lines contain 'q' integers representing the elements of the second matrix.",
   'output_format': "If the matrices are compatible for multiplication, display the resulting matrix with 'm' rows and 'q' columns.  If the matrices are not compatible, display an appropriate error message. If memory allocation fails, display a 'Memory allocation failed' message.",
   'sample_input': '2 3\n1 2 3\n4 5 6\n3 2\n7 8\n9 10\n11 12',
   'sample_output': '58 64\n139 154'}

code = """
#include <stdio.h>
#include <stdlib.h>

// Function to allocate a 2D matrix dynamically
int** allocate_matrix(int rows, int cols) {
    int** matrix = (int**)malloc(rows * sizeof(int*));
    if (matrix == NULL) return NULL;

    for (int i = 0; i < rows; i++) {
        matrix[i] = (int*)malloc(cols * sizeof(int));
        if (matrix[i] == NULL) return NULL;
    }
    return matrix;
}

// Function to free a dynamically allocated 2D matrix
void free_matrix(int** matrix, int rows) {
    for (int i = 0; i < rows; i++)
        free(matrix[i]);
    free(matrix);
}

int main() {
    int r1, c1, r2, c2;

    // Input dimensions
    printf("Enter rows and columns for Matrix A: ");
    scanf("%d %d", &r1, &c1);

    printf("Enter rows and columns for Matrix B: ");
    scanf("%d %d", &r2, &c2);

    // Check if multiplication is possible
    if (c1 != r2) {
        printf("Error: Incompatible dimensions for matrix multiplication.\n");
        return 1;
    }

    // Allocate matrices
    int** A = allocate_matrix(r1, c1);
    int** B = allocate_matrix(r2, c2);
    int** C = allocate_matrix(r1, c2);  // Result matrix

    if (A == NULL || B == NULL || C == NULL) {
        printf("Memory allocation failed.\n");
        return 1;
    }

    // Input elements for Matrix A
    printf("Enter elements of Matrix A:\n");
    for (int i = 0; i < r1; i++)
        for (int j = 0; j < c1; j++)
            scanf("%d", &A[i][j]);

    // Input elements for Matrix B
    printf("Enter elements of Matrix B:\n");
    for (int i = 0; i < r2; i++)
        for (int j = 0; j < c2; j++)
            scanf("%d", &B[i][j]);

    // Matrix multiplication
    for (int i = 0; i < r1; i++) {
        for (int j = 0; j < c2; j++) {
            C[i][j] = 0;
            for (int k = 0; k < c1; k++) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }

    // Print result matrix
    printf("Resultant Matrix (A x B):\n");
    for (int i = 0 i < r1; i++) {
        for (int j = 0; j < c2; j++)
            printf("%d ", C[i][j]);
        printf("\n");
    }

    // Free allocated memory
    free_matrix(A, r1);
    free_matrix(B, r2);
    free_matrix(C, r1);

    return 0;
}

"""

In [13]:
get_hint(question,code,'''main.c: In function ‘main’:
main.c:73:20: error: expected ‘,’ or ‘;’ before ‘i’
   73 |     for (int i = 0 i < r1; i++) {''')

{'hint_1': 'Look carefully at the line where the error occurs. Is there something missing in the syntax of the for loop?',
 'hint_2': "In C, when declaring a loop variable inside a for loop, you need to make sure the syntax is correct. Check if you're missing any necessary punctuation.",
 'hint_3': "When declaring the loop variable 'i' in the for loop, a semicolon is missing between the initialization and the condition. It should be 'for (int i = 0; i < r1; i++)'."}

In [ ]:
def check_code(question_details: dict, student_code: str, output: str):
    prompt = f"""
    You are a programming instructor evaluating a student's submission. Your job is to check if the student's code fulfills all the requirements of the original question, and if the output is correct.

    Below is the full information you have:

    --- Original Question ---
    {question_details}

    --- Student's Submitted Code ---
    {student_code}

    --- Output Produced by the Code ---
    {output}

    --- Evaluation Instructions ---
    1. Read the question carefully. Check for all required parts:
        - What the function is supposed to do.
        - Required input format (e.g., number of arguments, data types).
        - Expected output format (e.g., type and structure of return value).

    2. Check the student code:
        - Does it define the function with the correct name and number/types of parameters?
        - Does the logic correctly solve the problem as stated?
        - Does it return (not just print) the correct result in the correct format?

    3. Validate the output:
        - Calculate the expected output based on the arguments used in the student's function call.
        - Compare that with the provided output.
        - If the result is mathematically incorrect or improperly formatted, do NOT approve.

    --- Return Your Final Judgment in the Following JSON Format ---
    
        "Approved": (int) 1 if everything is correct, else 0,
        "Reason": (str) Briefly explain why it's approved or what’s wrong (e.g., wrong return type, incorrect logic, wrong function name, etc.)
    
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt,generation_config=genai.GenerationConfig(response_mime_type="application/json"))
    raw_text = response.text
    cleaned_json = clean_json_response(raw_text)

    try:
        json_data = json.loads(cleaned_json)
        # If Gemini wraps the content in a top-level "LabQuestions" key, unwrap it:
        return json_data
    except Exception as e:
        print("Parsing Error:", e)
        print("Raw output:", raw_text)
        return None

In [29]:
question = {'number': 1,
   'title': 'Simple Function to Calculate the Area of a Rectangle',
   'problem_statement': 'Write a Python function that calculates the area of a rectangle given its length and width.',
   'input_format': 'The function should take two arguments: length and width (both floats).',
   'output_format': 'The function should return the area of the rectangle (float).',
   'sample_input': 'length = 5.0, width = 3.0',
   'sample_output': '15.0'}

code = """
def calculate_rectangle_area(length: float, width: float) -> float:
    return length * width

area = calculate_rectangle_area(45.0, 8.0)
print("Area of rectangle:", area)
"""

op = '32.0'
res = check_code(question,code,op)

In [30]:
res

{'Approved': 1,
 'Reason': "The student's code fulfills all requirements. The function 'calculate_rectangle_area' is correctly defined, takes the correct input types, and returns the correct output type.  The calculation is also correct (45.0 * 8.0 = 360.0). The printed output is not part of the evaluation; only the function's return value is evaluated."}

## report functions

In [58]:
from app import app
from models import db, AssignmentProgress, Question, Student
from sqlalchemy import func
import matplotlib.pyplot as plt
from collections import Counter

# Needed for using the db.session inside Jupyter
app.app_context().push()

def get_avg_time_for_assignment(assignment_id):
    result = (
        db.session.query(func.avg(AssignmentProgress.time))
        .filter(AssignmentProgress.assignment_id == assignment_id)
        .scalar()
    )
    
    # Return the average time or 0 if no entries
    return round(result, 2) if result is not None else 0


In [23]:
assignment_id = 'abcd2'  # Replace with your actual assignment ID
avg_time = get_avg_time_for_assignment(assignment_id)
print(f"Average time for assignment {assignment_id}: {avg_time} minutes")


Average time for assignment abcd2: 165.83333333333334 minutes


In [37]:
def get_question_attempt_distribution(assignment_id):
    with app.app_context():
        # Step 1: Get all progress entries for the assignment, grouped by student
        results = (
            db.session.query(AssignmentProgress.student_id, AssignmentProgress.solved_questions)
            .filter(AssignmentProgress.assignment_id == assignment_id)
            .all()
        )

        if not results:
            print(f"No progress found for assignment ID: {assignment_id}")
            return {}

        # Step 2: Count how many students attempted X number of questions
        distribution = Counter()

        for student_id, total_questions in results:
            distribution[total_questions] += 1

        # Return the distribution dictionary
        return dict(distribution)

In [38]:
get_question_attempt_distribution('abcd2')  # Replace with your actual assignment ID

{3: 2, 1: 1, 2: 2, 0: 1}

In [41]:
def get_assignment_time_distribution(assignment_id):
    with app.app_context():
        # Step 1: Get all progress entries for the assignment, grouped by student
        results = (
            db.session.query(AssignmentProgress.time)
            .filter(AssignmentProgress.assignment_id == assignment_id)
            .all()
        )

        if not results:
            print(f"No progress found for assignment ID: {assignment_id}")
            return {}

        # Return the distribution dictionary
        return [time[0] for time in results]

In [42]:
get_assignment_time_distribution('abcd2')

[100, 190, 195, 120, 190, 200]

In [51]:
def get_avg_score_for_assignment(assignment_id):
    result = (
        db.session.query(func.avg(AssignmentProgress.score))
        .filter(AssignmentProgress.assignment_id == assignment_id)
        .scalar()
    )
    
    # Return the average time or 0 if no entries
    return round(result, 2) if result is not None else 0

In [52]:
get_avg_score_for_assignment('abcd2')

60.83

In [47]:
def get_assignment_score_distribution(assignment_id):
    with app.app_context():
        # Step 1: Get all progress entries for the assignment, grouped by student
        results = (
            db.session.query(AssignmentProgress.score)
            .filter(AssignmentProgress.assignment_id == assignment_id)
            .all()
        )

        if not results:
            print(f"No progress found for assignment ID: {assignment_id}")
            return {}

        # Return the distribution dictionary
        return [score[0] for score in results]

In [48]:
get_assignment_score_distribution('abcd2')

[100.0, 33.0, 66.0, 100.0, 66.0, 0.0]

In [55]:
def class_analysis(assignment_id):
    avg_time = get_avg_time_for_assignment(assignment_id)
    question_distribution = get_question_attempt_distribution(assignment_id)
    time_distribution = get_assignment_time_distribution(assignment_id)
    avg_score = get_avg_score_for_assignment(assignment_id)
    score_distribution = get_assignment_score_distribution(assignment_id)
    return {"avg_time": avg_time, "question_distribution": question_distribution, "time_distribution": time_distribution, "avg_score": avg_score, "score_distribution": score_distribution}

In [56]:
class_analysis('abcd2')

{'avg_time': 165.83,
 'question_distribution': {3: 2, 1: 1, 2: 2, 0: 1},
 'time_distribution': [100, 190, 195, 120, 190, 200],
 'avg_score': 60.83,
 'score_distribution': [100.0, 33.0, 66.0, 100.0, 66.0, 0.0]}

In [59]:
def get_student_question_parent_time_distribution(student_name, assignment_id):
    with app.app_context():
        # Step 1: Get student_id from student name
        student = db.session.query(Student).filter(Student.name == student_name).first()

        if not student:
            print(f"Student with name {student_name} not found.")
            return {}

        # Step 2: Get all questions for the given assignment_id and student_id
        results = (
            db.session.query(Question.question_id, Question.parent_time)
            .filter(Question.assignment_id == assignment_id, Question.student_id == student.userid)
            .all()
        )

        if not results:
            print(f"No questions found for student {student_name} in assignment ID: {assignment_id}")
            return {}

        # Step 3: Create a dictionary with question_id as key and parent_time as value
        question_time_dict = {question_id: parent_time for question_id, parent_time in results}

        return question_time_dict


In [60]:
get_student_question_parent_time_distribution('NAVYA', 'abcd2')

{'q1': 20, 'q2': 10, 'q3': 10}

In [61]:
def get_student_question_followup_time_distribution(student_name, assignment_id):
    with app.app_context():
        # Step 1: Get student_id from student name
        student = db.session.query(Student).filter(Student.name == student_name).first()

        if not student:
            print(f"Student with name {student_name} not found.")
            return {}

        # Step 2: Get all questions for the given assignment_id and student_id
        results = (
            db.session.query(Question.question_id, Question.followup_time)
            .filter(Question.assignment_id == assignment_id, Question.student_id == student.userid)
            .all()
        )

        if not results:
            print(f"No questions found for student {student_name} in assignment ID: {assignment_id}")
            return {}

        # Step 3: Create a dictionary with question_id as key and parent_time as value
        question_time_dict = {question_id: follow_time for question_id, follow_time in results}

        return question_time_dict


In [62]:
get_student_question_followup_time_distribution('NAVYA', 'abcd2')

{'q1': 20, 'q2': 15, 'q3': 25}

In [63]:
def get_student_question_attempt_distribution(student_name, assignment_id):
    with app.app_context():
        # Step 1: Get student_id from student name
        student = db.session.query(Student).filter(Student.name == student_name).first()

        if not student:
            print(f"Student with name {student_name} not found.")
            return {}

        # Step 2: Get all questions for the given assignment_id and student_id
        results = (
            db.session.query(Question.question_id, Question.parent_att)
            .filter(Question.assignment_id == assignment_id, Question.student_id == student.userid)
            .all()
        )

        if not results:
            print(f"No questions found for student {student_name} in assignment ID: {assignment_id}")
            return {}

        # Step 3: Create a dictionary with question_id as key and parent_time as value
        question_att_dict = {question_id: par_att for question_id, par_att in results}

        return question_att_dict


In [64]:
get_student_question_attempt_distribution('NAVYA', 'abcd2')

{'q1': 1, 'q2': 1, 'q3': 1}

In [66]:
def get_student_question_fol_attempt_distribution(student_name, assignment_id):
    with app.app_context():
        # Step 1: Get student_id from student name
        student = db.session.query(Student).filter(Student.name == student_name).first()

        if not student:
            print(f"Student with name {student_name} not found.")
            return {}

        # Step 2: Get all questions for the given assignment_id and student_id
        results = (
            db.session.query(Question.question_id, Question.followup_att)
            .filter(Question.assignment_id == assignment_id, Question.student_id == student.userid)
            .all()
        )

        if not results:
            print(f"No questions found for student {student_name} in assignment ID: {assignment_id}")
            return {}

        # Step 3: Create a dictionary with question_id as key and parent_time as value
        question_att_dict = {question_id: fol_att for question_id, fol_att in results}

        return question_att_dict


In [67]:
get_student_question_fol_attempt_distribution('NAVYA', 'abcd2')

{'q1': 2, 'q2': 1, 'q3': 1}

In [68]:
def student_analysis(student_name, assignment_id):
    question_parent_time = get_student_question_parent_time_distribution(student_name, assignment_id)
    question_followup_time = get_student_question_followup_time_distribution(student_name, assignment_id)
    question_parent_att = get_student_question_attempt_distribution(student_name, assignment_id)
    question_followup_att = get_student_question_fol_attempt_distribution(student_name, assignment_id)
    return {"question_parent_time": question_parent_time, "question_followup_time": question_followup_time, "question_parent_att": question_parent_att, "question_followup_att": question_followup_att}

In [69]:
student_analysis('NAVYA', 'abcd2')

{'question_parent_time': {'q1': 20, 'q2': 10, 'q3': 10},
 'question_followup_time': {'q1': 20, 'q2': 15, 'q3': 25},
 'question_parent_att': {'q1': 2, 'q2': 1, 'q3': 1},
 'question_followup_att': {'q1': 2, 'q2': 1, 'q3': 1}}